In [ ]:
!pip install -q transformers accelerate decord torch torchvision facenet-pytorch
!pip install -q opencv-python-headless


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.6 MB/s eta 0:00:00


In [ ]:
from google.colab import files
import os

os.makedirs("videos", exist_ok=True)

uploaded = files.upload()
for filename in uploaded.keys():
    os.rename(filename, f"videos/{filename}")

print("✅ Uploaded videos:", list(os.listdir("videos")))


Saving videoplayback (1).mp4 to videoplayback (1).mp4
Saving videoplayback.mp4 to videoplayback.mp4
✅ Uploaded videos: ['videoplayback (1).mp4', 'videoplayback.mp4']


In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel
from facenet_pytorch import InceptionResnetV1, MTCNN

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load XCLIP
xclip_model = CLIPModel.from_pretrained("microsoft/xclip-base-patch32").to(device)
xclip_processor = CLIPProcessor.from_pretrained("microsoft/xclip-base-patch32")

# Load FaceNet + face detector
face_model = InceptionResnetV1(pretrained="vggface2").eval().to(device)
mtcnn = MTCNN(keep_all=True, device=device)

print("✅ Models loaded on:", device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

You are using a model of type xclip to instantiate a model of type clip. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/786M [00:00<?, ?B/s]

Some weights of CLIPModel were not initialized from the model checkpoint at microsoft/xclip-base-patch32 and are newly initialized: ['vision_model.pre_layrnorm.bias', 'vision_model.pre_layrnorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

  0%|          | 0.00/107M [00:00<?, ?B/s]

✅ Models loaded on: cpu


In [ ]:
import numpy as np
from decord import VideoReader, cpu
from PIL import Image

# Prompts
actions = ["a person speaking", "a person dancing", "a person singing", "a person painting"]
emotions = ["a person happy", "a person sad", "a person angry", "a person neutral"]

def predict_with_all_scores(frames, prompts):
    """Return all prompts with confidence scores."""
    images = [Image.fromarray(f).resize((224,224)) for f in frames]
    inputs = xclip_processor(text=prompts, images=images, return_tensors="pt", padding=True).to(device)

    with torch.no_grad():
        logits_per_image = xclip_model(**inputs).logits_per_image
        probs = logits_per_image.softmax(dim=1).mean(dim=0)

    scores = [(prompts[i], float(probs[i])) for i in range(len(prompts))]
    return sorted(scores, key=lambda x: x[1], reverse=True)


In [ ]:
import os

os.makedirs("faces_debug", exist_ok=True)

def extract_face_embeddings(frames, video_name, chunk_id):
    """Detect faces, return embeddings, and save crops for debugging."""
    embeddings = []
    persons = []
    frame_count = 0

    for f in frames:
        img = Image.fromarray(f)
        boxes, probs = mtcnn.detect(img)
        if boxes is not None:
            faces = mtcnn(img)
            if faces is not None:
                with torch.no_grad():
                    emb = face_model(faces.to(device)).cpu().numpy()
                for j, e in enumerate(emb):
                    embeddings.append(e)
                    persons.append((video_name, chunk_id))

                    # Clamp box coordinates to avoid invalid crops
                    f_np = np.array(img)
                    h, w, _ = f_np.shape
                    x1, y1, x2, y2 = boxes[j].astype(int)
                    x1, y1 = max(0, x1), max(0, y1)
                    x2, y2 = min(w, x2), min(h, y2)

                    if x2 > x1 and y2 > y1:  # only save valid crops
                        crop = f_np[y1:y2, x1:x2]
                        crop_path = f"faces_debug/{video_name}_chunk{chunk_id}_frame{frame_count}_face{j}.jpg"
                        cv2.imwrite(crop_path, cv2.cvtColor(crop, cv2.COLOR_RGB2BGR))
        frame_count += 1
    return embeddings, persons


In [ ]:
import pandas as pd

def analyze_video(video_path, chunks=5, frames_per_chunk=16):
    vr = VideoReader(video_path, ctx=cpu(0))
    num_frames = len(vr)
    chunk_size = num_frames // chunks

    results = []
    for i in range(chunks):
        start = i * chunk_size
        end = min((i+1) * chunk_size, num_frames)
        indices = np.linspace(start, end-1, frames_per_chunk, dtype=int)
        frames = vr.get_batch(indices).asnumpy()

        # XCLIP
        action_scores = predict_with_all_scores(frames, actions)
        emotion_scores = predict_with_all_scores(frames, emotions)

        # Face embeddings + crops
        embeddings, persons = extract_face_embeddings(frames, os.path.basename(video_path), i+1)

        results.append({
            "Video": os.path.basename(video_path),
            "Chunk": i+1,
            "Frames": f"{start}-{end}",
            "Actions": ", ".join([f"{a}:{s:.2f}" for a,s in action_scores]),
            "Emotions": ", ".join([f"{e}:{s:.2f}" for e,s in emotion_scores]),
            "Embeddings": embeddings,
            "Embeddings_meta": persons
        })
    return results


In [ ]:
all_video_results = []

for video in os.listdir("videos"):
    video_path = os.path.join("videos", video)
    print(f"📽️ Processing {video}...")
    res = analyze_video(video_path, chunks=5, frames_per_chunk=16)
    all_video_results.extend(res)

print("✅ Finished all videos")


📽️ Processing videoplayback (1).mp4...
📽️ Processing videoplayback.mp4...
✅ Finished all videos


In [ ]:
from sklearn.cluster import DBSCAN

all_embeddings = []
meta_info = []

for r in all_video_results:
    for e, meta in zip(r["Embeddings"], r["Embeddings_meta"]):
        all_embeddings.append(e)
        meta_info.append(meta)

all_embeddings = np.array(all_embeddings)

# DBSCAN clustering (stricter threshold)
clustering = DBSCAN(eps=0.6, min_samples=2, metric="cosine").fit(all_embeddings)
labels = clustering.labels_

print("Clusters found:", np.unique(labels))
print("Counts per cluster:", np.bincount(labels[labels >= 0]))


Clusters found: [-1  0  1  2]
Counts per cluster: [66  2  3]


In [ ]:
person_counter = 0
idx = 0
for r in all_video_results:
    r["Persons"] = []
    for e in r["Embeddings"]:
        label = labels[idx]
        if label == -1:
            r["Persons"].append("Unknown")
        else:
            r["Persons"].append(f"Person_{label}")
        idx += 1


In [ ]:
final_results = []
for r in all_video_results:
    final_results.append({
        "Video": r["Video"],
        "Chunk": r["Chunk"],
        "Frames": r["Frames"],
        "Actions": r["Actions"],
        "Emotions": r["Emotions"],
        "Persons": ", ".join(set(r["Persons"])) if r["Persons"] else "No face detected"
    })

import pandas as pd
df_final = pd.DataFrame(final_results)
df_final.to_csv("video_identity_action_emotion.csv", index=False)

from google.colab import files
files.download("video_identity_action_emotion.csv")

df_final


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Video,Chunk,Frames,Actions,Emotions,Persons
0,videoplayback (1).mp4,1,0-808,"a person dancing:0.67, a person speaking:0.14,...","a person happy:0.78, a person neutral:0.14, a ...","Unknown, Person_1, Person_0"
1,videoplayback (1).mp4,2,808-1616,"a person dancing:0.62, a person speaking:0.16,...","a person happy:0.80, a person neutral:0.14, a ...","Unknown, Person_1, Person_0"
2,videoplayback (1).mp4,3,1616-2424,"a person dancing:0.63, a person speaking:0.16,...","a person happy:0.78, a person neutral:0.15, a ...",Person_0
3,videoplayback (1).mp4,4,2424-3232,"a person dancing:0.65, a person speaking:0.15,...","a person happy:0.78, a person neutral:0.15, a ...",Person_0
4,videoplayback (1).mp4,5,3232-4040,"a person dancing:0.69, a person speaking:0.12,...","a person happy:0.75, a person neutral:0.16, a ...",Person_0
5,videoplayback.mp4,1,0-716,"a person dancing:0.69, a person speaking:0.13,...","a person happy:0.78, a person neutral:0.14, a ...",Person_0
6,videoplayback.mp4,2,716-1432,"a person dancing:0.67, a person speaking:0.14,...","a person happy:0.81, a person neutral:0.14, a ...","Unknown, Person_0"
7,videoplayback.mp4,3,1432-2148,"a person dancing:0.64, a person speaking:0.15,...","a person happy:0.81, a person neutral:0.13, a ...",Person_0
8,videoplayback.mp4,4,2148-2864,"a person dancing:0.68, a person speaking:0.14,...","a person happy:0.80, a person neutral:0.14, a ...","Person_0, Person_2"
9,videoplayback.mp4,5,2864-3580,"a person dancing:0.68, a person speaking:0.13,...","a person happy:0.78, a person neutral:0.15, a ...",Person_0
